<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introducción

#Armado de ambiente

In [1]:
print('codigo para armado de ambiente')

codigo para armado de ambiente


#Desarrollo

In [2]:
print('codigo para el desarrollo de la practica')

codigo para el desarrollo de la practica


#Tabla de pasos

In [3]:
print('seccion de tabla de pasos')

seccion de tabla de pasos


#Conclusiones

#Bibliografía